In [1]:
#%pip install --extra-index-url https://test.pypi.org/simple ../../EnergyPlus-OOEP/
#%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
#%pip install ipywidgets tqdm gymnasium torch ray[rllib]

In [2]:
import logging as _logging_
_logging_.basicConfig(level='INFO')
#from energyplus.ooep.addons.logging import LogProvider

from energyplus.ooep.addons.display import ProgressProvider

from energyplus.ooep import (
    Simulator,
    Model,
    Weather,
    Report,
)

from energyplus.dataset.basic import dataset as _epds_

simulator = Simulator().add(
    ProgressProvider(),
    #LogProvider(),
)

await simulator.awaitable.run_forever(
    input=Simulator.InputSpecs(
        model=Model().open(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=Weather().open(_epds_.weathers / 'USA_FL_Tampa.Intl.AP.722110_TMY3.epw'),
    ),
    output=Simulator.OutputSpecs(
        report=Report().open('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    options=Simulator.RuntimeOptions(
        #design_day=True,
    ),
)

  0%|          | 0/100 [00:00<?, ?it/s]

<Task pending name='Task-6' coro=<Engine.run_forever() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:22>>

In [3]:
import numpy as _numpy_
import gymnasium as _gymnasium_

from energyplus.ooep.addons.rl.gymnasium.spaces import VariableBox
from energyplus.ooep.addons.rl.ray import SimulatorEnv

from energyplus.ooep.components.variables import (
    Actuator,
    OutputVariable,
)

from ray.rllib.algorithms.ppo import PPOConfig


config = (
    PPOConfig.from_dict({        
        'create_env_on_local_worker': True
    })
    .resources(num_gpus=1)
    .environment(
        SimulatorEnv, 
        env_config=SimulatorEnv.Config(
            action_space=_gymnasium_.spaces.Dict({
                'thermostat': VariableBox(
                    low=15., high=16.,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(Actuator.Ref(
                    type='Zone Temperature Control',
                    control_type='Heating Setpoint',
                    key='CORE_MID',
                ))
            }),    
            observation_space=_gymnasium_.spaces.Dict({
                'temperature': VariableBox(
                    low=-_numpy_.inf, high=+_numpy_.inf,
                    dtype=_numpy_.float32,
                    shape=(),
                ).bind(OutputVariable.Ref(
                    type='People Air Temperature',
                    key='CORE_MID',
                )),
            }),
            reward_function=lambda __x: 1,
            event_refs=[
                'begin_zone_timestep_after_init_heat_balance',
            ],
            simulator_factory=lambda simulator=simulator: simulator,
        )
    )
    .rollouts(
        #num_rollout_workers=10,
        num_rollout_workers=0,
        enable_connectors=False,
    )
    .framework("torch")
    .training(
        model={"fcnet_hiddens": [64, 64]},
        lr=0.0001,
    )
    .evaluation(
        #evaluation_interval=1,
        #evaluation_num_workers=0
    )
)


algo = config.build(use_copy=True)

/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:500: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/user@AD/lab/EnergyPlus-OOEP/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
2024-04-02 10:00:15,894	WARNING unified.py:56 -- Could not instantiate JsonLogger: cannot pickle '_hashlib.HMAC' objec

In [4]:
# force simulator to rerun to make the newly requested variables available
simulator.stop()
# start training
for _ in range(10):
    print(algo.train())

end None None
start None
end dbd78229be2f46c4891ecd77516a3f6e {'temperature': array(25.865873, dtype=float32)}
start None
end 3f6b781765a64b99b9e9f2e4e5eae8b8 {'temperature': array(25.865873, dtype=float32)}
start None


2024-04-02 10:00:24,491	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 0.00010000000000000003, 'total_loss': 9.937240221167123, 'policy_loss': -0.06315793074987909, 'vf_loss': 10.0, 'vf_explained_var': 0.7461372037087718, 'kl': 0.0019907165066093335, 'entropy': 1.4226238213559632, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_e

In [ ]:
algo.evaluate()

{'evaluation': {'sampler_results': {'episode_reward_max': 14501.0,
   'episode_reward_min': 14501.0,
   'episode_reward_mean': 14501.0,
   'episode_len_mean': 14502.0,
   'episode_media': {},
   'episodes_this_iter': 3,
   'policy_reward_min': {},
   'policy_reward_max': {},
   'policy_reward_mean': {},
   'custom_metrics': {},
   'hist_stats': {'episode_reward': [14501.0, 14501.0, 14501.0],
    'episode_lengths': [14502, 14502, 14502]},
   'sampler_perf': {'mean_raw_obs_processing_ms': 0.11099811089359796,
    'mean_inference_ms': 0.9705301517516801,
    'mean_action_processing_ms': 0.08443163993421014,
    'mean_env_wait_ms': 1.6712596755419582,
    'mean_env_render_ms': 0.0},
   'num_faulty_episodes': 0,
   'connector_metrics': {}},
  'episode_reward_max': 14501.0,
  'episode_reward_min': 14501.0,
  'episode_reward_mean': 14501.0,
  'episode_len_mean': 14502.0,
  'episode_media': {},
  'episodes_this_iter': 3,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_me

Exception ignored on calling ctypes callback function: <function EventManager._ep_callback_setters.<locals>.<dictcomp>.<lambda>.<locals>.<lambda> at 0x7f813410b2e0>
Traceback (most recent call last):
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 72, in <lambda>
    state, lambda _: trigger(StateEvent(ref=ref))
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 49, in trigger
    raise e
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/components/events.py", line 46, in trigger
    try: self.trigger(*args, **kwargs)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/events.py", line 43, in trigger
    return self._handlers[event.ref](event, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user@AD/lab/EnergyPlus-OOEP/packages/e

In [ ]:
simulator.stop()